In [1]:
#получение обучающей, тестовой и тестовой выборки
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

In [2]:
loaded_model_away_poly = joblib.load('model_poly_away_new.joblib')
loaded_model_home_poly = joblib.load('model_poly_home_new.joblib')

In [3]:
scaler_filename_home = "scaler_home_new.save"
scaler_filename_away = "scaler_away_new.save"
scaler_home = joblib.load(scaler_filename_home)
scaler_away = joblib.load(scaler_filename_away)

In [4]:
loaded_model_home_neuron = load_model('home_model_new.h5')
loaded_model_away_neuron = load_model('away_model_new.h5')

In [5]:
match_data = pd.read_csv('match_data.csv',index_col='id')

In [6]:
match_data['points'] = match_data.apply(lambda row: 3 if row['goals'] > match_data[(match_data['fixtureID'] == row['fixtureID']) & (match_data['team_id'] != row['team_id'])]['goals'].max() else (0 if row['goals'] < match_data[(match_data['fixtureID'] == row['fixtureID']) & (match_data['team_id'] != row['team_id'])]['goals'].max() else 1), axis=1)

In [7]:
def create_samples(data):
    x = []
    for _, row in data.iterrows():
        new_line = []
        team_data = match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-10::]
        home_data = team_data[team_data['position'] == 'home']
        away_data = team_data[team_data['position'] == 'away']
        points_last = match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-5::]['points'].sum()
        print(match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-5::])
        new_line.extend([row['team_id'], row['name'] ,row['time'], home_data['goals'].mean(), away_data['goals'].mean(), home_data['shots'].mean(), away_data['shots'].mean(), home_data['xg'].mean(), away_data['xg'].mean(), points_last])

        x.extend([new_line]) 
        #y.extend([[1, row['home_goals']], [2, row['away_goals']]])
    return x

In [8]:
import http.client
import json

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "55d8d0822bdb3026f52b479e85e3a447"
    }

conn.request("GET", f"/fixtures?next=8&league=235", headers=headers)
res = conn.getresponse()
data = res.read()
json_data = json.loads(data)
conn.request("GET", f"/fixtures?next=10&league=39", headers=headers)
res = conn.getresponse()
data = res.read()
json_data_2 = json.loads(data)
conn.request("GET", f"/fixtures?next=9&league=78", headers=headers)
res = conn.getresponse()
data = res.read()
json_data_3 = json.loads(data)
conn.request("GET", f"/fixtures?next=10&league=135", headers=headers)
res = conn.getresponse()
data = res.read()
json_data_4 = json.loads(data)
if 'response' in json_data and json_data['response']:
    json_data = json_data['response']

if 'response' in json_data_2 and json_data_2['response']:
    json_data_2 = json_data_2['response']

if 'response' in json_data_3 and json_data_3['response']:
    json_data_3 = json_data_3['response']

if 'response' in json_data_4 and json_data_4['response']:
    json_data_4 = json_data_4['response']

json_data = json_data + json_data_2 + json_data_3 + json_data_4
#json_data = json_data_2


In [9]:
home_teams = []
away_teams = []
for fixture in json_data:
    home_teams.append([fixture['teams']['home']['id'], fixture['fixture']['timestamp'], fixture['teams']['home']['name']])
    away_teams.append([fixture['teams']['away']['id'], fixture['fixture']['timestamp'], fixture['teams']['away']['name']])
home_teams = pd.DataFrame(home_teams, columns=['team_id', 'time', 'name'])
away_teams = pd.DataFrame(away_teams, columns=['team_id', 'time', 'name'])
home_teams

,team_id,time,name
0,596,1709424000,Zenit Saint Petersburg
1,779,1709424000,FC Rostov
2,621,1709424000,Krasnodar
3,555,1709424000,CSKA Moscow
4,1088,1709424000,Dinamo Moscow
5,1085,1709424000,Akhmat Grozny
6,2006,1709424000,Baltika
7,2011,1709424000,Nizhny Novgorod
8,65,1702670400,Nottingham Forest
9,35,1702738800,Bournemouth


In [10]:
away_teams

,team_id,time,name
0,558,1709424000,Spartak Moscow
1,1079,1709424000,Krylya Sovetov
2,1083,1709424000,Rubin
3,1080,1709424000,Orenburg
4,597,1709424000,Lokomotiv Moscow
5,1084,1709424000,Ural
6,2012,1709424000,PFC Sochi
7,1993,1709424000,Fakel Voronezh
8,47,1702670400,Tottenham
9,1359,1702738800,Luton


In [11]:
#формирование x
x_home = create_samples(home_teams)
x_home = pd.DataFrame(x_home, columns=['team_id','name', 'time', 'mean_goals_home', 'mean_goals_away', 'shots_home', 'shots_away', 'xg_home', 'xg_away', 'points'])
x_away = create_samples(away_teams)
x_away = pd.DataFrame(x_away, columns=['team_id','name','time', 'mean_goals_home', 'mean_goals_away', 'shots_home', 'shots_away', 'xg_home', 'xg_away', 'points'])
x_home

      fixtureID  team_id position  goals  shots   xg        time  points
id                                                                      
4522    1042126      596     home    1.0    4.0  1.0  1699720200       1
4570    1042134      596     home    3.0    7.0  3.0  1700914500       3
4645    1141321      596     away    0.0    0.0  0.0  1701270900       0
4687    1042145      596     away    1.0    3.0  1.0  1701610200       0
4778    1042150      596     home    1.0    8.0  1.0  1702140300       3
      fixtureID  team_id position  goals  shots   xg        time  points
id                                                                      
4555    1042127      779     away    1.0    6.0  1.0  1699806600       0
4624    1042135      779     home    1.0    3.0  1.0  1701014400       3
4646    1141325      779     home    1.0    3.0  1.0  1701270900       1
4683    1042143      779     away    0.0    5.0  0.0  1701601200       0
4757    1042154      779     away    0.0    1.0  0.

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points
0,596,Zenit Saint Petersburg,1709424000,1.833333,1.250000,4.833333,2.500000,1.833333,1.250000,7
1,779,FC Rostov,1709424000,1.600000,0.800000,4.400000,4.400000,1.600000,0.800000,5
2,621,Krasnodar,1709424000,2.200000,1.400000,4.600000,5.200000,2.252000,1.400000,10
3,555,CSKA Moscow,1709424000,2.200000,0.800000,5.600000,3.400000,2.200000,0.800000,5
4,1088,Dinamo Moscow,1709424000,1.333333,1.750000,4.500000,6.000000,1.378333,1.750000,13
5,1085,Akhmat Grozny,1709424000,0.800000,0.800000,4.200000,2.800000,0.800000,0.800000,5
6,2006,Baltika,1709424000,1.285714,0.000000,3.714286,1.000000,1.587143,0.166667,3
7,2011,Nizhny Novgorod,1709424000,1.000000,0.500000,2.000000,1.166667,0.807500,0.500000,4
8,65,Nottingham Forest,1702670400,1.400000,0.600000,3.800000,2.800000,1.292000,0.948000,1
9,35,Bournemouth,1702738800,1.600000,1.800000,6.600000,4.800000,1.444000,1.568000,13


In [12]:
x_away

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points
0,558,Spartak Moscow,1709424000,2.200000,0.800000,6.600000,3.000000,2.200000,0.634000,9
1,1079,Krylya Sovetov,1709424000,2.750000,0.666667,7.750000,2.666667,2.750000,0.645000,7
2,1083,Rubin,1709424000,0.500000,0.833333,4.250000,3.500000,0.500000,1.023333,12
3,1080,Orenburg,1709424000,1.000000,0.666667,6.250000,3.666667,1.000000,0.666667,5
4,597,Lokomotiv Moscow,1709424000,1.833333,1.750000,5.000000,5.000000,1.978333,1.830000,11
5,1084,Ural,1709424000,1.500000,0.833333,5.000000,3.000000,1.472500,0.833333,3
6,2012,PFC Sochi,1709424000,1.000000,1.000000,4.500000,3.750000,1.000000,1.642500,2
7,1993,Fakel Voronezh,1709424000,1.200000,0.800000,3.800000,3.800000,1.264000,0.800000,6
8,47,Tottenham,1702670400,1.833333,1.750000,7.500000,2.750000,2.098333,1.060000,4
9,1359,Luton,1702738800,1.333333,1.000000,3.166667,2.500000,0.781667,0.707500,3


In [13]:
elo_england_22 = pd.read_csv('./elo/csv/England/2022/ratingElo2022_2023PL.csv')
elo_england_23 = pd.read_csv('./elo/csv/England/2023/ratingElo2023_2024PL.csv')

elo_england_22.set_index('id', inplace=True)
elo_england_23.set_index('id', inplace=True)

elo_england_23['prev_elos'] = elo_england_23.apply(lambda row: elo_england_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_england_22.index else row['prev_elos'], axis=1)
elo_england_23.reset_index(inplace=True)

elo_russian_22 = pd.read_csv('./elo/csv/Russia/2022/ratingElo2022_2023RFPL.csv')
elo_russian_23 = pd.read_csv('./elo/csv/Russia/2023/ratingElo2023_2024RFPL.csv')

elo_russian_22.set_index('id', inplace=True)
elo_russian_23.set_index('id', inplace=True)

elo_russian_23['prev_elos'] = elo_russian_23.apply(lambda row: elo_russian_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_russian_22.index else row['prev_elos'], axis=1)
elo_russian_23.reset_index(inplace=True)

elo_bundes_22 = pd.read_csv('./elo/csv/Germany/2022/ratingElo2022_2023Bundes.csv')
elo_bundes_23 = pd.read_csv('./elo/csv/Germany/2023/ratingElo2023_2024Bundes.csv')

elo_bundes_22.set_index('id', inplace=True)
elo_bundes_23.set_index('id', inplace=True)

elo_bundes_23['prev_elos'] = elo_bundes_23.apply(lambda row: elo_bundes_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_bundes_22.index else row['prev_elos'], axis=1)
elo_bundes_23.reset_index(inplace=True)

elo_italy_22 = pd.read_csv('./elo/csv/Italy/2022/ratingElo2022_2023Italy.csv')
elo_italy_23 = pd.read_csv('./elo/csv/Italy/2023/ratingElo2023_2024Italy.csv')

elo_italy_22.set_index('id', inplace=True)
elo_italy_23.set_index('id', inplace=True)

elo_italy_23['prev_elos'] = elo_italy_23.apply(lambda row: elo_italy_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_italy_22.index else row['prev_elos'], axis=1)
elo_italy_23.reset_index(inplace=True)

all_elo = pd.concat([elo_england_23, elo_russian_23, elo_bundes_23, elo_italy_23])
all_elo.reset_index(inplace=True)

In [14]:
import re, json
def process_row(row):
    return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])
def add_column_elo(frame):
    frame['elo'] = frame.apply(process_row, axis=1)

In [15]:
samples = [x_home, x_away]
for item in samples:
    add_column_elo(item)

C:\Users\arbol\AppData\Local\Temp\ipykernel_18592\1540769638.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])
C:\Users\arbol\AppData\Local\Temp\ipykernel_18592\1540769638.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])


In [16]:
x_home['diff_elo'] = x_home['elo'] - x_away['elo']
x_away['diff_elo'] = x_away['elo'] - x_home['elo']

In [17]:
x_home

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points,elo,diff_elo
0,596,Zenit Saint Petersburg,1709424000,1.833333,1.250000,4.833333,2.500000,1.833333,1.250000,7,1785,125
1,779,FC Rostov,1709424000,1.600000,0.800000,4.400000,4.400000,1.600000,0.800000,5,1607,14
2,621,Krasnodar,1709424000,2.200000,1.400000,4.600000,5.200000,2.252000,1.400000,10,1767,134
3,555,CSKA Moscow,1709424000,2.200000,0.800000,5.600000,3.400000,2.200000,0.800000,5,1719,145
4,1088,Dinamo Moscow,1709424000,1.333333,1.750000,4.500000,6.000000,1.378333,1.750000,13,1741,-4
5,1085,Akhmat Grozny,1709424000,0.800000,0.800000,4.200000,2.800000,0.800000,0.800000,5,1559,41
6,2006,Baltika,1709424000,1.285714,0.000000,3.714286,1.000000,1.587143,0.166667,3,1525,68
7,2011,Nizhny Novgorod,1709424000,1.000000,0.500000,2.000000,1.166667,0.807500,0.500000,4,1525,-50
8,65,Nottingham Forest,1702670400,1.400000,0.600000,3.800000,2.800000,1.292000,0.948000,1,1745,-204
9,35,Bournemouth,1702738800,1.600000,1.800000,6.600000,4.800000,1.444000,1.568000,13,1866,156


In [18]:
x_home_predict = x_home.drop(['time', 'elo', 'name'], axis=1)
x_away_predict = x_away.drop(['time', 'elo', 'name'], axis=1)

In [19]:
x_home_predict = scaler_home.transform(x_home_predict)
x_away_predict = scaler_away.transform(x_away_predict)

In [20]:
poly_home_value = loaded_model_home_poly.predict(x_home_predict)
poly_away_value = loaded_model_away_poly.predict(x_away_predict)

In [21]:
print("Полином:")
for index, row in x_home.iterrows():
    print(f"{row['name']} - {x_away.iloc[index]['name']}: {round(poly_home_value[index], 3)}:{round(poly_away_value[index], 3)}")

Полином:
Zenit Saint Petersburg - Spartak Moscow: 1.202:1.055
FC Rostov - Krylya Sovetov: 1.125:1.177
Krasnodar - Rubin: 1.419:0.861
CSKA Moscow - Orenburg: 1.339:1.002
Dinamo Moscow - Lokomotiv Moscow: 1.137:1.404
Akhmat Grozny - Ural: 0.967:1.051
Baltika - PFC Sochi: 1.148:1.104
Nizhny Novgorod - Fakel Voronezh: 0.75:1.041
Nottingham Forest - Tottenham: 0.947:1.314
Bournemouth - Luton: 1.371:0.731
Chelsea - Sheffield Utd: 1.548:0.846
Manchester City - Crystal Palace: 1.951:0.939
Newcastle - Fulham: 1.554:1.175
Burnley - Everton: 0.93:1.434
Arsenal - Brighton: 1.438:1.087
Brentford - Aston Villa: 1.295:1.548
West Ham - Wolves: 1.303:1.161
Liverpool - Manchester United: 1.614:0.976
Borussia Monchengladbach - Werder Bremen: 1.056:1.073
FSV Mainz 05 - FC Heidenheim: 1.015:1.067
VfL BOCHUM - Union Berlin: 1.202:1.274
FC Augsburg - Borussia Dortmund: 0.969:1.492
SV Darmstadt 98 - VfL Wolfsburg: 0.821:1.374
RB Leipzig - 1899 Hoffenheim: 1.788:0.985
SC Freiburg - FC Koln: 1.276:1.314
Bayer L

In [22]:
nueron_home_value = loaded_model_home_neuron.predict(x_home_predict)
neuron_away_value = loaded_model_away_neuron.predict(x_away_predict)

2/2 [==============================] - 0s 4ms/step


In [23]:
print("Нейронная сеть:")
for index, row in x_home.iterrows():
    home_goals = round(float(nueron_home_value[index][0]),3)
    away_goals = round(float(neuron_away_value[index][0]), 3)
    print(f"{row['name']} - {x_away.iloc[index]['name']}: {home_goals}:{away_goals}")

Нейронная сеть:
Zenit Saint Petersburg - Spartak Moscow: 1.699:1.119
FC Rostov - Krylya Sovetov: 1.549:1.257
Krasnodar - Rubin: 1.828:0.902
CSKA Moscow - Orenburg: 1.817:1.022
Dinamo Moscow - Lokomotiv Moscow: 1.576:1.33
Akhmat Grozny - Ural: 1.409:1.107
Baltika - PFC Sochi: 1.22:1.115
Nizhny Novgorod - Fakel Voronezh: 0.974:1.061
Nottingham Forest - Tottenham: 1.243:1.426
Bournemouth - Luton: 1.974:0.862
Chelsea - Sheffield Utd: 2.176:0.858
Manchester City - Crystal Palace: 2.437:0.941
Newcastle - Fulham: 1.853:1.242
Burnley - Everton: 1.288:1.315
Arsenal - Brighton: 1.851:1.221
Brentford - Aston Villa: 1.646:1.393
West Ham - Wolves: 1.661:1.183
Liverpool - Manchester United: 2.222:1.067
Borussia Monchengladbach - Werder Bremen: 1.785:1.129
FSV Mainz 05 - FC Heidenheim: 1.454:1.097
VfL BOCHUM - Union Berlin: 1.614:1.191
FC Augsburg - Borussia Dortmund: 1.234:1.441
SV Darmstadt 98 - VfL Wolfsburg: 1.363:1.334
RB Leipzig - 1899 Hoffenheim: 2.168:1.125
SC Freiburg - FC Koln: 1.626:1.213
